Read in file
==

In [1]:
import pandas as pd
pd.options.display.max_columns = 99
five_rows = pd.read_csv("loans_2007.csv", nrows=5)
#print(five_rows)
thous_rows = pd.read_csv("loans_2007.csv", nrows=3000)
thous_rows.memory_usage().sum()/(1024*1024)

1.1902618408203125

Exploring The Data In Chunk
==
**How many columns have a numeric type? How many have a string type?**

In [2]:
chunks = pd.read_csv("loans_2007.csv", chunksize=1000)
for c in chunks:
    print(c.dtypes.value_counts())

float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64   

**How many unique values are there in each string column?**

In [3]:
chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
col_count = {}
combined_count = {}
rows = 0
for c in chunks:
    str_cols = c.select_dtypes(include=['object'])
    rows += len(str_cols)
    for s in str_cols.columns:
        chunk_col_count = str_cols[s].value_counts()
        if s in col_count:
            col_count[s].append(chunk_col_count)
        else:
            col_count[s] = [chunk_col_count]

for s in col_count:
    combined_count[s] = pd.concat(col_count[s])
for key in combined_count.keys():
    print(key,":",len(combined_count[key]))

verification_status : 43
home_ownership : 52
last_pymnt_d : 911
term : 27
earliest_cr_line : 5537
loan_status : 58
last_credit_pull_d : 1078
id : 3538
grade : 105
emp_length : 177
sub_grade : 522
pymnt_plan : 16
addr_state : 653
zip_code : 8117
purpose : 201
application_type : 15
int_rate : 1124
emp_title : 36960
issue_d : 113
revol_util : 13590
initial_list_status : 15
title : 26924


**Which float columns have no missing values and could be candidates for conversion to the integer type?**

In [4]:
chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
is_null_float_dict = {}
for c in chunks:
    is_null_float = c.select_dtypes(include=["float"]).isnull().sum()
    for col in is_null_float.index:
        if col in is_null_float_dict:
            is_null_float_dict[col] += is_null_float[col]
        else:
            is_null_float_dict[col] = is_null_float[col]
print(is_null_float_dict)
num_cols_five = pd.read_csv('loans_2007.csv', usecols=is_null_float_dict.keys(), nrows=5)
print(num_cols_five)

{'dti': 3, 'pub_rec_bankruptcies': 1368, 'acc_now_delinq': 32, 'collection_recovery_fee': 3, 'funded_amnt_inv': 3, 'total_pymnt': 3, 'loan_amnt': 3, 'total_acc': 32, 'member_id': 3, 'recoveries': 3, 'pub_rec': 32, 'out_prncp_inv': 3, 'last_pymnt_amnt': 3, 'open_acc': 32, 'collections_12_mths_ex_med': 148, 'delinq_2yrs': 32, 'policy_code': 3, 'annual_inc': 7, 'chargeoff_within_12_mths': 148, 'revol_bal': 3, 'tax_liens': 108, 'total_pymnt_inv': 3, 'out_prncp': 3, 'funded_amnt': 3, 'inq_last_6mths': 32, 'total_rec_int': 3, 'delinq_amnt': 32, 'installment': 3, 'total_rec_late_fee': 3, 'total_rec_prncp': 3}
   member_id  loan_amnt  funded_amnt  funded_amnt_inv  installment  \
0  1296599.0     5000.0       5000.0           4975.0       162.87   
1  1314167.0     2500.0       2500.0           2500.0        59.83   
2  1313524.0     2400.0       2400.0           2400.0        84.33   
3  1277178.0    10000.0      10000.0          10000.0       339.31   
4  1311748.0     3000.0       3000.0    

**Calculate the total memory usage across all of the chunks**

In [5]:
chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
usage = 0
for c in chunks:
    usage += c.memory_usage().sum()/(1024**2)
print(usage)

16.8772354126


Optimizing String Columns
==

**Inspect the string columns**

In [6]:
string_five_rows = pd.read_csv("loans_2007.csv", usecols = combined_count.keys(), nrows=5)
print(string_five_rows)

        id        term int_rate grade sub_grade                 emp_title  \
0  1077501   36 months   10.65%     B        B2                       NaN   
1  1077430   60 months   15.27%     C        C4                     Ryder   
2  1077175   36 months   15.96%     C        C5                       NaN   
3  1076863   36 months   13.49%     C        C1       AIR RESOURCES BOARD   
4  1075358   60 months   12.69%     B        B5  University Medical Group   

  emp_length home_ownership verification_status   issue_d  loan_status  \
0  10+ years           RENT            Verified  Dec-2011   Fully Paid   
1   < 1 year           RENT     Source Verified  Dec-2011  Charged Off   
2  10+ years           RENT        Not Verified  Dec-2011   Fully Paid   
3  10+ years           RENT     Source Verified  Dec-2011   Fully Paid   
4     1 year           RENT     Source Verified  Dec-2011      Current   

  pymnt_plan         purpose                 title zip_code addr_state  \
0          n     c

*Observation:*

*1.  Some columns can be converted to a numeric type after cleaning (e.g., term, int_rate, revol_util).*

*2. Some can be converted to the category type (e.g., the string columns which contain few unique values).*

**Which string columns contain less than 5% unique values?**

In [7]:
less_half_unique_list = []
for key in combined_count.keys():
    per_unique = len(combined_count[key])/rows
    if per_unique < 0.05:
        less_half_unique_list.append(key)
print(less_half_unique_list)

['verification_status', 'home_ownership', 'last_pymnt_d', 'term', 'loan_status', 'last_credit_pull_d', 'grade', 'emp_length', 'sub_grade', 'pymnt_plan', 'addr_state', 'purpose', 'application_type', 'int_rate', 'issue_d', 'initial_list_status']


**Read as chunk while convert to catogorical, convert to datetime**

In [8]:
to_cat_list = ['grade', 'sub_grade', 'emp_length', 'home_ownership', 
'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 
'addr_state','initial_list_status', 'application_type']

In [9]:
to_date_list = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]

In [19]:
convert_cat = {}
for col in to_cat_list:
    convert_cat[col] = "category"
print(convert_cat)

#improve_chunks = pd.read_csv('loans_2007.csv', dtype=convert_cat, parse_dates=to_date_list, chunksize=3000)
improve_chunks = pd.read_csv('loans_2007.csv', dtype=convert_cat, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"], chunksize=3000)

{'verification_status': 'category', 'home_ownership': 'category', 'pymnt_plan': 'category', 'addr_state': 'category', 'loan_status': 'category', 'application_type': 'category', 'emp_length': 'category', 'grade': 'category', 'purpose': 'category', 'initial_list_status': 'category', 'sub_grade': 'category'}


**Convert some string columns to numerical, and downcast numerical columns**

In [20]:
per_to_float = ['int_rate', 'revol_util']
improve_usage = 0
for c in improve_chunks:
    for col in per_to_float:
        c[col] = pd.to_numeric(c[col].str.rstrip("%"), downcast="float")
    c['term'] = pd.to_numeric(c['term'].str.lstrip(" ").str.rstrip(" months"), downcast="integer")
    improve_usage += c.memory_usage().sum()/(1024**2)
print(improve_usage)

test_usage = 0
for c in improve_chunks:
    test_usage += c.memory_usage().sum()/(1024**2)

print(test_usage)

13.1828212738
0


*Observation:*

*The overall memory usage used to be 16.87 MB, now it's 13.18 MB*

*We will be able to save more space by downcast others numerical columns*

**Next Step: convert others numerical columns to a more space efficient subtype**